# HuggingFace로 기계번역 구현하기

이번에는 HuggingFace로 기계번역 모델을 학습해 볼 것입니다.
먼저 필요한 library들을 설치하고 import합시다.

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn sacrebleu sentencepiece

In [ ]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer

## Dataset 준비

기계번역을 위한 dataset을 준비하겠습니다.
Dataset은 `Helsinki-NLP/opus-100`을 활용합니다.

In [ ]:
books = load_dataset("Helsinki-NLP/opus-100", "en-ko")
books["train"][1]

{'translation': {'en': "I ain't fishing' 'em out.", 'ko': '그거 꺼내려다가는'}}

보시다시피 각 data는 영어 문장을 `en`에, 한국어 문장을 `ko`에 저장하고 있습니다.

이번에는 tokenizer를 불러와 data를 미리 tokenize 하겠습니다.

In [ ]:
source_lang = "en"
target_lang = "ko"
prefix = "translate English to Korean: "
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

def preprocess_function(data):
    inputs = [prefix + text[source_lang] for text in data["translation"]]
    targets = [text[target_lang] for text in data["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

기계번역과 같이 입력과 출력을 모두 tokenize하는 경우에는 출력 text를 `tokenizer`의 `text_target`인자로 넘겨주면 됩니다. 이처럼 기계 번역 dataset을 불러오는 것은 imdb dataset과 별반 다르지 않습니다.

마지막으로 `data_collator`를 다음과 같이 구현합니다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='google-t5/t5-small')

## Model 구현

이번에는 기계번역을 위한 sequence-to-sequence 모델을 구현하겠습니다.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Sequence-to-sequence 모델은 위와 같이 `AutoModelForSeq2SeqLM`을 활용하면 됩니다.
Pre-trained 모델로 `google-t5/t5-small`이라는 것을 사용하고 있습니다.
이 코드 하나로 우리는 기계 번역과 같은 문제를 풀 수 있는 encoder-decoder 구조의 모델을 구현할 수 있습니다.

## 학습

마지막으로 학습 코드를 구현하겠습니다.
학습은 감정 분석과 똑같이 `training_args`를 정의하고 `Trainer`로 이전에 구현한 것들을 모두 수합하면 됩니다.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="hf_mt",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,  # 첫 주차 때 배운 weight decay를 조절하는 hyper-parameter입니다.
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True  # 실제로 평가를 진행할 때는 내부적으로 정의된 beam search 등을 활용하여 text를 생성합니다.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

마지막으로 학습하면 다음과 같습니다.

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,8.898300
1000,8.187200
1500,8.095000
2000,8.036400
2500,7.970700
3000,7.939400
3500,7.907700
4000,7.867400
4500,7.866100
5000,7.839700


TrainOutput(global_step=62500, training_loss=7.57248191015625, metrics={'train_runtime': 3897.8165, 'train_samples_per_second': 256.554, 'train_steps_per_second': 16.035, 'total_flos': 1.0393154930343936e+16, 'train_loss': 7.57248191015625, 'epoch': 1.0})

학습 결과를 보면 다음과 같습니다.

In [ ]:
from transformers import pipeline

text = "translate English to Korean: Hello! My name is Seunghyuk Cho."
translator = pipeline("translation", model='./hf_mt', max_new_tokens=128, device='cuda', tokenizer=tokenizer)
translator(text)

[{'translation_text': '-의 그 그 그 그 그 그 그 그 그 그 그 그...................................................'}]

보시다시피 성능이 그렇게 좋지 않습니다. 아마 더 많은 학습 시간을 요구하는 듯 합니다.

하지만 중요한 것은 data부터 모델, 학습 코드 구현까지 매우 쉽게 했다는 것입니다.